In [ ]:
# Side Notes/ideas
# Paients that have died due to the cancer can measure the severity through the grade of tumor, metastasis at diagnosis, prior malignancy, last known disease status, and time from diagnosis to death.
# In the current dataset there is a bias towards patients that are dead, in the testing dataset need to ensure that there is a balance of alive and dead patients
# Use this NN to classify severity level of cancer in alive patients
# as input the NN will take the features mentioned above and output a severity level from 1-5 (1 being least severe, 5 being most severe)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import numpy as np

# ============== 1. DEFINE THE NEURAL NETWORK ==============
class CancerSeverityPredictor(nn.Module):
    def __init__(self, input_size):
        super(CancerSeverityPredictor, self).__init__()
        
        # Architecture: input -> 64 -> 32 -> 16 -> 1
        self.layer1 = nn.Linear(input_size, 64)
        self.bn1 = nn.BatchNorm1d(64)
        self.dropout1 = nn.Dropout(0.3)
        
        self.layer2 = nn.Linear(64, 32)
        self.bn2 = nn.BatchNorm1d(32)
        self.dropout2 = nn.Dropout(0.3)
        
        self.layer3 = nn.Linear(32, 16)

        self.bn3 = nn.BatchNorm1d(16)
        self.dropout3 = nn.Dropout(0.2)
        
        self.output = nn.Linear(16, 1)
        
        # Weight initialization
        self._init_weights()
    
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        # Layer 1
        x = self.layer1(x)
        x = self.bn1(x)
        x = torch.relu(x)
        x = self.dropout1(x)
        
        # Layer 2
        x = self.layer2(x)
        x = self.bn2(x)
        x = torch.relu(x)
        x = self.dropout2(x)
        
        # Layer 3
        x = self.layer3(x)
        x = self.bn3(x)
        x = torch.relu(x)
        x = self.dropout3(x)
        
        # Output (no activation for regression)
        x = self.output(x)
        return x

# ============== 2. PREPARE DATA ==============
# Load medical dataset
def load_medical_data():
    try:
        # Load the actual CSV file
        csv_path = r'C:\Users\Meghaj Kabra\Desktop\AI Health 2026 Competition\output.csv'
        df = pd.read_csv(csv_path, low_memory=False)
        print(f"✓ Loaded data: {df.shape}")
        
        feature_mapping = [
        'demographic.gender': 'gender',
        'demographic.race': 'race',
        'demographic.ethnicity': 'ethnicity',
        'diagnoses.age_at_diagnosis': 'age_at_diagnosis',
        'demographic.vital_status': 'vital_status',
        'diagnoses.tumor_grade': 'tumor_grade',
        'diagnoses.morphology': 'morphology',
        'diagnoses.site_of_resection_or_biopsy': 'site_of_biopsy',
        'diagnoses.laterality': 'laterality',
        'diagnoses.prior_malignancy': 'prior_malignancy',
        'diagnoses.prior_treatment': 'prior_treatment',
        'diagnoses.synchronous_malignancy': 'another_malignancy',
        'exposures.alcohol_history': 'alcohol_history',
        'exposures.alcohol_intensity': 'alcohol_intensity',
        'exposures.cigarettes_per_day': 'tobacco_frequency',
        'exposures.tobacco_smoking_onset_year': 'tobacco_onset',
        'diagnoses.metastasis_at_diagnosis': 'metastasis',
        'diagnoses.last_known_disease_status': 'disease_status',
        'diagnoses.progression_or_recurrence': 'progression',
        'demographic.days_to_death': 'days_to_death',
        'follow_ups.karnofsky_performance_status': 'karnofsky_score',
        'diagnoses.who_cns_grade': 'who_grade']
        
        features = list(feature_mapping.keys())
        # Check which columns exist
        available_cols = [col for col in feature_mapping if col in df.columns]
        print(f"✓ Found {len(available_cols)} features: {available_cols[:5]}...")
        
        # Extract features
        X_df = df[available_cols].copy()
        
        # Target: vital_status (encode as 0=Alive, 1=Dead)
        if 'demographic.vital_status' in df.columns:
            y = (df['demographic.vital_status'] == 'Dead').astype(int).values
        else:
            print("⚠ Warning: No target column found, using synthetic data")
            y = np.random.randint(0, 2, len(df))
        
        # Encode categorical features
        for col in X_df.columns:
            if X_df[col].dtype == 'object':
                le = LabelEncoder()
                X_df[col] = le.fit_transform(X_df[col].fillna('Unknown'))
        
        # Fill missing numeric values
        X_df = X_df.fillna(X_df.median())
        
        X = X_df.values.astype(np.float32)
        
        print(f"✓ Processed shape: X={X.shape}, y={y.shape}")
        return X, y
        
    except Exception as e:
        print(f"✗ Error loading data: {e}")
        print("⚠ Using synthetic data for demonstration")
        np.random.seed(42)
        X = np.random.rand(1000, 10)
        y = np.random.randint(0, 2, 1000)
        return X, y


X, y = load_medical_data()

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# Convert to PyTorch tensors
X_train = torch.FloatTensor(X_train)
y_train = torch.FloatTensor(y_train).reshape(-1, 1)
X_test = torch.FloatTensor(X_test)
y_test = torch.FloatTensor(y_test).reshape(-1, 1)

# ============== 3. SETUP TRAINING ==============
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n🖥️ Using device: {device}")

model = CancerSeverityPredictor(input_size=X_train.shape[1]).to(device)
criterion = nn.BCEWithLogitsLoss()  # Binary cross-entropy for classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Move data to device
X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)

# ============== 4. TRAINING LOOP ==============
num_epochs = 100
batch_size = 32

print("\n📊 Training Cancer Severity Prediction Model...")
for epoch in range(num_epochs):
    model.train()
    
    # Mini-batch training
    permutation = torch.randperm(X_train.size()[0])
    epoch_loss = 0
    num_batches = 0
    
    for i in range(0, X_train.size()[0], batch_size):
        indices = permutation[i:i+batch_size]
        batch_x, batch_y = X_train[indices], y_train[indices]
        
        # Skip batch if size is 1 (BatchNorm issue)
        if batch_x.size(0) == 1:
            continue
        
        # Forward pass
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        
        # Backward pass
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
        epoch_loss += loss.item()
        num_batches += 1
    
    # Print progress
    if (epoch + 1) % 10 == 0:
        avg_loss = epoch_loss / num_batches if num_batches > 0 else 0
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

# ============== 5. EVALUATION ==============
model.eval()
with torch.no_grad():
    # Get predictions
    train_pred = torch.sigmoid(model(X_train))
    test_pred = torch.sigmoid(model(X_test))
    
    # Calculate accuracy
    train_acc = ((train_pred > 0.5).float() == y_train).float().mean()
    test_acc = ((test_pred > 0.5).float() == y_test).float().mean()

print(f"\n✅ Results:")
print(f"Training Accuracy: {train_acc*100:.2f}%")
print(f"Test Accuracy: {test_acc*100:.2f}%")

# Show sample predictions
print(f"\n🔍 Sample Predictions (first 5 test samples):")
for i in range(min(5, len(y_test))):
    actual = "Dead" if y_test[i].item() == 1 else "Alive"
    predicted = "Dead" if test_pred[i].item() > 0.5 else "Alive"
    confidence = test_pred[i].item() * 100
    print(f"  Actual: {actual:5} | Predicted: {predicted:5} | Confidence: {confidence:.1f}%")

# ============== 6. SAVE MODEL ==============
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scaler': scaler,
    'input_size': X_train.shape[1]
}, 'cancer_severity_model.pth')
print(f"\n💾 Model saved as 'cancer_severity_model.pth'")

✓ Loaded data: (21634, 164)
✓ Found 10 features: ['demographic.gender', 'demographic.race', 'demographic.ethnicity', 'demographic.age_at_index', 'primary_site']...
✓ Processed shape: X=(21634, 10), y=(21634,)

🖥️ Using device: cuda

📊 Training Cancer Severity Prediction Model...
Epoch [10/100], Loss: 0.4020
Epoch [20/100], Loss: 0.3887
Epoch [30/100], Loss: 0.3831
Epoch [40/100], Loss: 0.3796
Epoch [50/100], Loss: 0.3716
Epoch [60/100], Loss: 0.3714
Epoch [70/100], Loss: 0.3698
Epoch [80/100], Loss: 0.3707
Epoch [90/100], Loss: 0.3692
Epoch [100/100], Loss: 0.3686

✅ Results:
Training Accuracy: 85.98%
Test Accuracy: 85.67%

🔍 Sample Predictions (first 5 test samples):
  Actual: Dead  | Predicted: Dead  | Confidence: 83.6%
  Actual: Dead  | Predicted: Dead  | Confidence: 92.0%
  Actual: Dead  | Predicted: Dead  | Confidence: 81.5%
  Actual: Dead  | Predicted: Dead  | Confidence: 89.7%
  Actual: Dead  | Predicted: Dead  | Confidence: 92.9%

💾 Model saved as 'cancer_severity_model.pth'
